Prediction and clustering task

In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, rand_score
from src.embedding import new_HeatGeo
import scanpy as sc
import numpy as np
import phate
import pandas as pd
import os
from omegaconf import OmegaConf

Results

In [106]:
# load results and check in config files the knn, tau, and emb_dim.
def load_results(path):
    results = pd.DataFrame()
    for root, dirs, files in os.walk(path):
        for filename in files:
            if filename.endswith("_0.pkl"):
                last_is_config = False
                dt_current = pd.read_pickle(os.path.join(root, filename))
                print(os.path.join(root, filename), "res")
            if filename.endswith("config.yaml") and not last_is_config:
                last_is_config = True
                config = OmegaConf.load(os.path.join(root, filename))
                print(os.path.join(root, filename), "config")
                dataset_name = config.dataset_name
                config = config.model
                order = 0 if "order" not in config else config.order
                order = 0 if order is None else order
                tau = 0 if "tau" not in config.keys() else config.tau
                knn = 0 if "knn" not in config.keys() else config.knn
                n_neighbors = 0 if "n_neighbors" not in config.keys() else config.n_neighbors
                emb_dim = 0 if "emb_dim" not in config.keys() else config.emb_dim
                n_components = 0 if "n_components" not in config.keys() else config.n_components
                perplexity = 0 if "perplexity" not in config.keys() else config.perplexity
                min_dist = 0 if "min_dist" not in config.keys() else config.min_dist
                denoising = 0 if "denoising" not in config.keys() else config.denoising

                # print(order, tau, knn, emb_dim, n_neighbors, n_components)
                dt_current["order"] = order
                dt_current["tau"] = tau
                dt_current["knn"] = knn
                dt_current["emb_dim"] = emb_dim
                dt_current["n_neighbors"] = n_neighbors
                dt_current["n_components"] = n_components
                dt_current["dataset_name"] = dataset_name
                dt_current["perplexity"] = perplexity
                dt_current["min_dist"] = min_dist
                dt_current["denoising"] = denoising
                results = pd.concat([results,dt_current])
    return results
 

On PBMC (not very good)

In [20]:
res = load_results("/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/multiruns/pred")

/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/multiruns/pred/2023-04-10_19-00-44/0/pred_PBMC_PCA_0.pkl res
/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/multiruns/pred/2023-04-10_19-00-44/0/.hydra/config.yaml config
/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/multiruns/pred/2023-04-10_19-00-44/1/pred_PBMC_PCA_0.pkl res
/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/multiruns/pred/2023-04-10_19-00-44/1/.hydra/config.yaml config
/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/multiruns/pred/2023-04-10_19-00-44/2/pred_PBMC_PCA_0.pkl res
/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/multiruns/pred/2023-04-10_19-00-44/2/.hydra/config.yaml config
/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/multiruns/pred/2023-04-10_19-02-31/6/pred_PBMC_t-SNE_0.pkl res
/netw

In [26]:
res.Method == "diff_map"
res_d = res[res.Method == "diff_map"]
res_d

,Method,Seed,Accuracy (t),Rand Score (t),Adjusted Rand Score (t),Homogeneity,Completeness,V-measure,Adjusted Rand Score,Adjusted Mutual Info Score,...,order,tau,knn,emb_dim,n_neighbors,n_components,dataset_name,perplexity,min_dist,denoising
0,diff_map,42,0.752009,0.809700,0.566754,0.549752,0.483735,0.514635,0.290112,0.510506,...,None,10.0,50,2,0,0,PBMC,0,0.0,0
1,diff_map,43,0.754305,0.806524,0.563983,0.557062,0.497310,0.525493,0.303853,0.521439,...,None,10.0,50,2,0,0,PBMC,0,0.0,0
2,diff_map,44,0.745121,0.783119,0.530353,0.555333,0.488657,0.519866,0.312299,0.515798,...,None,10.0,50,2,0,0,PBMC,0,0.0,0
3,diff_map,45,0.718714,0.790361,0.519508,0.561886,0.495788,0.526772,0.307014,0.522685,...,None,10.0,50,2,0,0,PBMC,0,0.0,0
4,diff_map,46,0.748565,0.803011,0.563248,0.556635,0.496093,0.524623,0.314036,0.520561,...,None,10.0,50,2,0,0,PBMC,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,diff_map,47,0.607348,0.665360,0.324802,0.280575,0.273944,0.277220,0.118844,0.270799,...,None,50.0,50,10,0,0,PBMC,0,0.0,0
6,diff_map,48,0.619977,0.651322,0.328166,0.288986,0.254166,0.270460,0.130735,0.264319,...,None,50.0,50,10,0,0,PBMC,0,0.0,0
7,diff_map,49,0.631458,0.685543,0.364025,0.287797,0.248884,0.266930,0.129507,0.260754,...,None,50.0,50,10,0,0,PBMC,0,0.0,0
8,diff_map,50,0.613088,0.659493,0.329300,0.284148,0.272556,0.278231,0.130053,0.271890,...,None,50.0,50,10,0,0,PBMC,0,0.0,0


In [23]:
res_mean = res.groupby(["Method" ,"order", "tau", "knn", "emb_dim", "n_neighbors", "n_components"]).mean().round(3).reset_index()
res_std = res.groupby(["Method" ,"order", "tau", "knn", "emb_dim", "n_neighbors", "n_components"]).std().round(3).reset_index()

In [24]:
res_mean.Method.unique()

array(['PCA', 'UMAP', 'heat_geo_euler', 'heat_geo_mar', 'phate', 't-SNE'],
      dtype=object)

In [19]:
print(res_mean.to_latex(escape=False))

/tmp/ipykernel_9266/1785560721.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(res_mean.to_latex(escape=False))


\begin{tabular}{llrlrrrrrrrrrrrrrrrrr}
\toprule
{} &          Method &  order &   tau &  knn &  emb_dim &  n_neighbors &  n_components &  denoising &  Accuracy (t) &  Rand Score (t) &  Adjusted Rand Score (t) &  Homogeneity &  Completeness &  V-measure &  Adjusted Rand Score &  Adjusted Mutual Info Score &  Silhouette Score &  time(s) &  perplexity &  min_dist \\
\midrule
0   &             PCA &      0 &   0.0 &    0 &        0 &            0 &             2 &          0 &         0.896 &           0.930 &                    0.822 &        0.767 &         0.637 &      0.696 &                0.554 &                       0.694 &             0.433 &    0.046 &         0.0 &       0.0 \\
1   &             PCA &      0 &   0.0 &    0 &        0 &            0 &             5 &          0 &         0.946 &           0.958 &                    0.893 &        0.839 &         0.838 &      0.838 &                0.852 &                       0.837 &             0.493 &    0.050 &         0.0 & 

On the swiss roll and tree

In [98]:
res = load_results("/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/multiruns/pred_toy")
res_mean = res.groupby(["Method" ,"order", "tau", "knn", "emb_dim", "n_neighbors", "n_components", "dataset_name", "min_dist", "perplexity", "denoising"]).mean().round(3).reset_index()
res_std = res.groupby(["Method" ,"order", "tau", "knn", "emb_dim", "n_neighbors", "n_components", "dataset_name", "min_dist", "perplexity", "denoising"]).std().round(3).reset_index()

/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/multiruns/pred_toy/2023-04-11_21-31-58/0/pred_tree_PCA_0.pkl res
/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/multiruns/pred_toy/2023-04-11_21-31-58/0/.hydra/config.yaml config
/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/multiruns/pred_toy/2023-04-11_21-31-58/1/pred_swiss_roll_PCA_0.pkl res
/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/multiruns/pred_toy/2023-04-11_21-31-58/1/.hydra/config.yaml config
/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/multiruns/pred_toy/2023-04-11_21-32-24/5/pred_swiss_roll_t-SNE_0.pkl res
/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/multiruns/pred_toy/2023-04-11_21-32-24/5/.hydra/config.yaml config
/network/scratch/g/guillaume.huguet/projects/metric_embeddings/logs/experiments/multiruns/pred_toy/2023-04-11_

In [99]:
res.Method.unique(), res_mean.Method.unique()

(array(['PCA', 't-SNE', 'diff_map', 'UMAP', 'heat_geo_euler',
        'heat_geo_mar', 'phate'], dtype=object),
 array(['PCA', 'UMAP', 'diff_map', 'heat_geo_euler', 'heat_geo_mar',
        'phate', 't-SNE'], dtype=object))

In [100]:
print(res_mean.to_latex(escape=False))

\begin{tabular}{llrlrrrrlrrrrrrrrrrrrr}
\toprule
{} &          Method &  order &   tau &  knn &  emb_dim &  n_neighbors &  n_components & dataset_name &  min_dist &  perplexity &  denoising &  Accuracy (t) &  Rand Score (t) &  Adjusted Rand Score (t) &  Homogeneity &  Completeness &  V-measure &  Adjusted Rand Score &  Adjusted Mutual Info Score &  Silhouette Score &  time(s) \\
\midrule
0  &             PCA &      0 &   0.0 &    0 &        0 &            0 &             2 &   swiss_roll &       0.0 &           0 &          0 &         0.984 &           0.985 &                    0.960 &        0.526 &         0.461 &      0.491 &                0.340 &                       0.489 &             0.412 &    0.002 \\
1  &             PCA &      0 &   0.0 &    0 &        0 &            0 &             2 &         tree &       0.0 &           0 &          0 &         0.784 &           0.863 &                    0.572 &        0.560 &         0.590 &      0.574 &                0.456 &      

/tmp/ipykernel_15660/1785560721.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(res_mean.to_latex(escape=False))


In [101]:
method_list = "PCA UMAP phate diff_map t-SNE heat_geo_mar".split()
knn_list = [0,20]
tau_list = [0,10, "auto"]
mask = (res["Method"].isin(method_list)) & (res["knn"].isin(knn_list)) & (res["n_neighbors"].isin(knn_list)) & (res["tau"].isin(tau_list))
res_small = res[mask].reset_index(drop=True)
res_t = res_small[res_small.dataset_name == "tree"]
res_sr = res_small[res_small.dataset_name == "swiss_roll"]


In [104]:
res_mean = res_t.groupby(["Method" ,"order", "tau", "knn", "emb_dim", "n_neighbors", "n_components", "dataset_name", "min_dist", "perplexity", "denoising"]).mean().round(3).reset_index()
res_std = res_t.groupby(["Method" ,"order", "tau", "knn", "emb_dim", "n_neighbors", "n_components", "dataset_name", "min_dist", "perplexity", "denoising"]).std().round(3).reset_index()
col_drop = "order tau knn emb_dim n_neighbors n_components dataset_name min_dist perplexity".split() + ["Rand Score (t)", "Adjusted Rand Score"]
res_mean = res_mean.drop(columns=col_drop, axis=1)
res_mean.set_index("Method", inplace=True)
res_std.set_index("Method", inplace=True)
res_std = res_std.drop(columns=col_drop, axis=1)
tab = res_mean.astype(str) + " $\pm$ " + res_std.astype(str)

In [105]:
print(tab.to_latex(escape=False))

\begin{tabular}{llllllllll}
\toprule
{} &  denoising &       Accuracy (t) & Adjusted Rand Score (t) &        Homogeneity &       Completeness &          V-measure & Adjusted Mutual Info Score &   Silhouette Score &             time(s) \\
Method       &            &                    &                         &                    &                    &                    &                            &                    &                     \\
\midrule
PCA          &  0 $\pm$ 0 &  0.784 $\pm$ 0.053 &       0.572 $\pm$ 0.088 &   0.56 $\pm$ 0.052 &    0.59 $\pm$ 0.04 &  0.574 $\pm$ 0.046 &          0.573 $\pm$ 0.046 &  0.457 $\pm$ 0.029 &   0.002 $\pm$ 0.002 \\
UMAP         &  0 $\pm$ 0 &  0.923 $\pm$ 0.022 &       0.822 $\pm$ 0.046 &  0.687 $\pm$ 0.046 &  0.709 $\pm$ 0.044 &  0.698 $\pm$ 0.045 &          0.697 $\pm$ 0.045 &  0.638 $\pm$ 0.057 &   5.542 $\pm$ 0.766 \\
UMAP         &  0 $\pm$ 0 &   0.92 $\pm$ 0.031 &       0.816 $\pm$ 0.062 &  0.686 $\pm$ 0.069 &  0.707 $\pm$ 0.061 &  0.

/tmp/ipykernel_15660/1405857127.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(tab.to_latex(escape=False))


rTesting the methods

In [2]:
pbmc = sc.datasets.pbmc3k_processed()
data = pbmc.X
labels = pbmc.obs["louvain"]
labels = labels.replace(to_replace=['CD4 T cells', 'CD14+ Monocytes', 'B cells', 'CD8 T cells', 'NK cells', 'FCGR3A+ Monocytes', 'Dendritic cells', 'Megakaryocytes'],
value=[0,1,2,3,4,5,6,7])
labels = np.array(labels)

In [3]:
data_train, data_test, label_train, label_test = train_test_split(data, labels, test_size=0.33, random_state=42)

In [5]:
op = new_HeatGeo(knn=30, emb_dim=10)
emb_train = op.fit_transform(data_train)
op = new_HeatGeo(knn=30, emb_dim=10)
emb_test = op.fit_transform(data_test)

Changed MDS solver to `smacof`, `sge` not implemented for 2<dim.
Changed MDS solver to `smacof`, `sge` not implemented for 2<dim.


In [6]:
knn_classifier = KNeighborsClassifier(n_neighbors=30)
knn_classifier.fit(emb_train, label_train)
pred = knn_classifier.predict(emb_test)
knn_classifier.score(emb_test, label_test)


0.8553386911595867

PHATE

In [10]:
op = phate.PHATE(knn=30, n_components=10, verbose=False)
emb_train = op.fit_transform(data_train)
op = phate.PHATE(knn=30, n_components=10, verbose=False)
emb_test = op.fit_transform(data_test)

In [11]:
knn_classifier = KNeighborsClassifier(n_neighbors=30)
knn_classifier.fit(emb_train, label_train)
pred = knn_classifier.predict(emb_test)
knn_classifier.score(emb_test, label_test)


0.8082663605051664

In [13]:
rand_score(label_test,pred)

0.8839199229317602

In [14]:
accuracy_score(label_test,pred)

0.8082663605051664

In [15]:
confusion_matrix(label_test,pred)

array([[355,   0,   2,  18,   0,   1,   0,   0],
       [  0, 165,   0,   0,   0,   6,   0,   0],
       [  5,   0, 100,   0,   0,   0,   1,   0],
       [ 35,   0,   0,  56,   0,   0,   0,   0],
       [ 14,   0,   0,  42,   0,   0,   0,   0],
       [  2,  25,   0,   0,   0,  27,   0,   0],
       [  2,   2,   2,   0,   0,   3,   1,   0],
       [  1,   0,   0,   0,   0,   6,   0,   0]])

In [38]:
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

def bench_k_means(kmeans, data, labels):
    """Benchmark to evaluate the KMeans initialization methods.

    Parameters
    ----------
    kmeans : KMeans instance
        A :class:`~sklearn.cluster.KMeans` instance with the initialization
        already set.
    name : str
        Name given to the strategy. It will be used to show the results in a
        table.
    data : ndarray of shape (n_samples, n_features)
        The data to cluster.
    labels : ndarray of shape (n_samples,)
        The labels used to compute the clustering metrics which requires some
        supervision.
    """
    estimator = make_pipeline(StandardScaler(), kmeans).fit(data)
    results = []

    # Define the metrics which require only the true labels and estimator
    # labels
    clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score,
    ]
    results += [m(labels, estimator[-1].labels_) for m in clustering_metrics]

    # The silhouette score requires the full dataset
    results += [
        metrics.silhouette_score(
            data,
            estimator[-1].labels_,
            metric="euclidean",
        )
    ]    
    return results

In [39]:
emb = new_HeatGeo(knn=15).fit_transform(data_train)

In [41]:
kmeans = KMeans(init="k-means++", n_clusters=8, n_init=10, random_state=42)
bench_k_means(kmeans, data=emb, labels=label_train)

[0.7072205849998836,
 0.5467763270314948,
 0.6167343318906441,
 0.36758189811006536,
 0.6136849995625326,
 0.4328958702625871]